# Proyecto de limpieza de datos
Autor: Iñaki Sebastián Orozco García    

Fecha: 24/11/2024

Para este proyecto usaré un dataset de BackBlaze una empresa de servidores que lleva años recabando la información S.M.A.R.T (Métricas estandarizadas de evaluación) de los discos duros de sus equipos con el atributo agregado Failure que indica si ese disco falló al día siguiente, este dataset es en el que he estado trabajando en mi proyecto de tesis para hacer predicciones de fallas con un árbol de decisión y una LSTM. 

Los 3 puntos en los que trabajaré en este proyecto giran en torno a un buen desempeño usando este set de datos en esos dos algoritmos (DT, LSTM)

- Imputación de datos faltantes             <- Elegida 
- Codificación de variables categóricas     <- Elegida
- Transformación de potencia
- Escalamiento                              <- Elegida
- Discretización

Esto es lo que se hará con las columnas del dataframe:

- date: Nada, nos sirve para la LSTM.
- serial_number: Variable categórica.
- model: Variable categórica, se va a dividir dos variables: Empresa y modelo.
- capacity_bytes: Variable categórica.
- failure: Nada.
---------
- datacenter: Eliminarla, no es reelevante para nosotros le sirve a la empresa que comparte el set de datos.
- cluster_id: Eliminarla, no es reelevante para nosotros le sirve a la empresa que comparte el set de datos.
- vault_id: Eliminarla, no es reelevante para nosotros le sirve a la empresa que comparte el set de datos.
- pod_id: Eliminarla, no es reelevante para nosotros le sirve a la empresa que comparte el set de datos.
- pod_slot_num: Eliminarla, no es reelevante para nosotros le sirve a la empresa que comparte el set de datos.
- is_legacy_format: Eliminarla, no es reelevante para nosotros le sirve a la empresa que comparte el set de datos.
--------

y también los smart que son de dos tipos:
- smart_normalized: PENDIENTE
- smart_raw: PENDIENTE

# Procesamiento inicial 

### Importar el set de datos en un dataframe de pandas

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Como nuestro dataset tiene muchos datos, definiré unas funciones para procesar los datos en chunks eliminando las columnas que no nos interesan sin cargarlas al df.

In [3]:
def process_chunks(file_path):
    chunksize = 10 ** 6
    dtype_dict = {'date': 'str', 'serial_number': 'str', 'model': 'str', 'capacity_bytes': 'int32', 'failure': 'bool', 'datacenter': 'str', 'cluster_id': 'int8', 'vault_id': 'int16', 'pod_id': 'int16', 'pod_slot_num': 'float32', 'is_legacy_format': 'bool', 'smart_1_normalized': 'float64', 'smart_1_raw': 'float64', 'smart_2_normalized': 'float64', 'smart_2_raw': 'float64', 'smart_3_normalized': 'float64', 'smart_3_raw': 'float64', 'smart_4_normalized': 'float64', 'smart_4_raw': 'float64', 'smart_5_normalized': 'float64', 'smart_5_raw': 'float64', 'smart_7_normalized': 'float64', 'smart_7_raw': 'float64', 'smart_8_normalized': 'float64', 'smart_8_raw': 'float64', 'smart_9_normalized': 'float64', 'smart_9_raw': 'float64', 'smart_10_normalized': 'float64', 'smart_10_raw': 'float64', 'smart_11_normalized': 'float64', 'smart_11_raw': 'float64', 'smart_12_normalized': 'float64', 'smart_12_raw': 'float64', 'smart_13_normalized': 'float64', 'smart_13_raw': 'float64', 'smart_15_normalized': 'float64', 'smart_15_raw': 'float64', 'smart_16_normalized': 'float64', 'smart_16_raw': 'float64', 'smart_17_normalized': 'float64', 'smart_17_raw': 'float64', 'smart_18_normalized': 'float64', 'smart_18_raw': 'float64', 'smart_22_normalized': 'float64', 'smart_22_raw': 'float64', 'smart_23_normalized': 'float64', 'smart_23_raw': 'float64', 'smart_24_normalized': 'float64', 'smart_24_raw': 'float64', 'smart_27_normalized': 'float64', 'smart_27_raw': 'float64', 'smart_71_normalized': 'float64', 'smart_71_raw': 'float64', 'smart_82_normalized': 'float64', 'smart_82_raw': 'float64', 'smart_90_normalized': 'float64', 'smart_90_raw': 'float64', 'smart_160_normalized': 'float64', 'smart_160_raw': 'float64', 'smart_161_normalized': 'float64', 'smart_161_raw': 'float64', 'smart_163_normalized': 'float64', 'smart_163_raw': 'float64', 'smart_164_normalized': 'float64', 'smart_164_raw': 'float64', 'smart_165_normalized': 'float64', 'smart_165_raw': 'float64', 'smart_166_normalized': 'float64', 'smart_166_raw': 'float64', 'smart_167_normalized': 'float64', 'smart_167_raw': 'float64', 'smart_168_normalized': 'float64', 'smart_168_raw': 'float64', 'smart_169_normalized': 'float64', 'smart_169_raw': 'float64', 'smart_170_normalized': 'float64', 'smart_170_raw': 'float64', 'smart_171_normalized': 'float64', 'smart_171_raw': 'float64', 'smart_172_normalized': 'float64', 'smart_172_raw': 'float64', 'smart_173_normalized': 'float64', 'smart_173_raw': 'float64', 'smart_174_normalized': 'float64', 'smart_174_raw': 'float64', 'smart_175_normalized': 'float64', 'smart_175_raw': 'float64', 'smart_176_normalized': 'float64', 'smart_176_raw': 'float64', 'smart_177_normalized': 'float64', 'smart_177_raw': 'float64', 'smart_178_normalized': 'float64', 'smart_178_raw': 'float64', 'smart_179_normalized': 'float64', 'smart_179_raw': 'float64', 'smart_180_normalized': 'float64', 'smart_180_raw': 'float64', 'smart_181_normalized': 'float64', 'smart_181_raw': 'float64', 'smart_182_normalized': 'float64', 'smart_182_raw': 'float64', 'smart_183_normalized': 'float64', 'smart_183_raw': 'float64', 'smart_184_normalized': 'float64', 'smart_184_raw': 'float64', 'smart_187_normalized': 'float64', 'smart_187_raw': 'float64', 'smart_188_normalized': 'float64', 'smart_188_raw': 'float64', 'smart_189_normalized': 'float64', 'smart_189_raw': 'float64', 'smart_190_normalized': 'float64', 'smart_190_raw': 'float64', 'smart_191_normalized': 'float64', 'smart_191_raw': 'float64', 'smart_192_normalized': 'float64', 'smart_192_raw': 'float64', 'smart_193_normalized': 'float64', 'smart_193_raw': 'float64', 'smart_194_normalized': 'float64', 'smart_194_raw': 'float64', 'smart_195_normalized': 'float64', 'smart_195_raw': 'float64', 'smart_196_normalized': 'float64', 'smart_196_raw': 'float64', 'smart_197_normalized': 'float64', 'smart_197_raw': 'float64', 'smart_198_normalized': 'float64', 'smart_198_raw': 'float64', 'smart_199_normalized': 'float64', 'smart_199_raw': 'float64', 'smart_200_normalized': 'float64', 'smart_200_raw': 'float64', 'smart_201_normalized': 'float64', 'smart_201_raw': 'float64', 'smart_202_normalized': 'float64', 'smart_202_raw': 'float64', 'smart_206_normalized': 'float64', 'smart_206_raw': 'float64', 'smart_210_normalized': 'float64', 'smart_210_raw': 'float64', 'smart_218_normalized': 'float64', 'smart_218_raw': 'float64', 'smart_220_normalized': 'float64', 'smart_220_raw': 'float64', 'smart_222_normalized': 'float64', 'smart_222_raw': 'float64', 'smart_223_normalized': 'float64', 'smart_223_raw': 'float64', 'smart_224_normalized': 'float64', 'smart_224_raw': 'float64', 'smart_225_normalized': 'float64', 'smart_225_raw': 'float64', 'smart_226_normalized': 'float64', 'smart_226_raw': 'float64', 'smart_230_normalized': 'float64', 'smart_230_raw': 'float64', 'smart_231_normalized': 'float64', 'smart_231_raw': 'float64', 'smart_232_normalized': 'float64', 'smart_232_raw': 'float64', 'smart_233_normalized': 'float64', 'smart_233_raw': 'float64', 'smart_234_normalized': 'float64', 'smart_234_raw': 'float64', 'smart_235_normalized': 'float64', 'smart_235_raw': 'float64', 'smart_240_normalized': 'float64', 'smart_240_raw': 'float64', 'smart_241_normalized': 'float64', 'smart_241_raw': 'float64', 'smart_242_normalized': 'float64', 'smart_242_raw': 'float64', 'smart_244_normalized': 'float64', 'smart_244_raw': 'float64', 'smart_245_normalized': 'float64', 'smart_245_raw': 'float64', 'smart_246_normalized': 'float64', 'smart_246_raw': 'float64', 'smart_247_normalized': 'float64', 'smart_247_raw': 'float64', 'smart_248_normalized': 'float64', 'smart_248_raw': 'float64', 'smart_250_normalized': 'float64', 'smart_250_raw': 'float64', 'smart_251_normalized': 'float64', 'smart_251_raw': 'float64', 'smart_252_normalized': 'float64', 'smart_252_raw': 'float64', 'smart_254_normalized': 'float64', 'smart_254_raw': 'float64', 'smart_255_normalized': 'float64', 'smart_255_raw': 'float64'}
    aggregated_result = pd.DataFrame() 

    for chunk in pd.read_csv(file_path, chunksize=chunksize, dtype=dtype_dict):
        chunk = delete_rows(chunk)
        
        aggregated_result = pd.concat([aggregated_result, chunk])

    return aggregated_result

def delete_rows(df):
    df.head()
    columns_to_delete = ['datacenter', 'cluster_id', 'vault_id', 'pod_id', 'pod_slot_num', 'is_legacy_format']
    smart_allowed = []
    df = df.drop(columns=columns_to_delete)
    return df


def getdata(folder_path):
    df = pd.DataFrame()
    for file_name in os.listdir(folder_path):
        # TODO REMOVE THIS
        if file_name.endswith(".csv") and file_name.startswith("2024-07-0"):
            file_path = os.path.join(folder_path, file_name)
            df = pd.concat([df, process_chunks(file_path)])
            print(file_path, ' done')
    print(df.columns)
    return df

Cargamos los datos en el df sin las columnas innecesarias

In [4]:
folder_path = '../data/external/data_Q3_2024/data_Q3_2024'

df = getdata(folder_path)

../data/external/data_Q3_2024/data_Q3_2024\2024-07-01.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-02.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-03.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-04.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-05.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-06.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-07.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-08.csv  done
../data/external/data_Q3_2024/data_Q3_2024\2024-07-09.csv  done
Index(['date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized',
       'smart_2_raw', 'smart_3_normalized',
       ...
       'smart_250_normalized', 'smart_250_raw', 'smart_251_normalized',
       'smart_251_raw', 'smart_252_normalized', 'smart_252_raw',
       'smart_254_normalized', 'smart_254_raw', 'smart_255_normalized',
       'smart_255_raw'],

### Analizamos el dataframe 

Generamos un varias gráficas de mapas de calor para visualizar el df y los atributos nulos que tiene.

In [5]:
# date, serial_number, model, capacity_bytes, failure
sub_dfs = [df.iloc[:, :5]]

# Smart Attributes
columns_for_each_plot = 25
sub_dfs += [df.iloc[:, i:i+columns_for_each_plot] for i in range(5, df.shape[1], columns_for_each_plot )]

heatmap_dir = '../reports/figures/heat_maps/'
os.makedirs(heatmap_dir, exist_ok=True)

for idx, sub_df in enumerate(sub_dfs):
    print(f"{idx+1}/{len(sub_dfs)}")
    sns.heatmap(sub_df.isnull(), cbar=False)
    plt.tight_layout()  # Adjust the layout to ensure nothing is cut off
    # plt.show()
    plt.savefig(f'../reports/figures/heat_maps/heat_map_{idx+1}.png')
    plt.close()

1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
9/9


Ahora eliminaremos las columnas que solo tienen datos faltantes.

In [12]:
nan_columns = df.columns[df.isna().all()].tolist()
print(f"Columns with only NaN values: {nan_columns}")

df.drop(columns=nan_columns, inplace=True)

Columns with only NaN values: ['smart_15_normalized', 'smart_15_raw', 'smart_160_normalized', 'smart_160_raw', 'smart_161_normalized', 'smart_161_raw', 'smart_163_normalized', 'smart_163_raw', 'smart_164_normalized', 'smart_164_raw', 'smart_176_normalized', 'smart_176_raw', 'smart_178_normalized', 'smart_178_raw', 'smart_250_normalized', 'smart_250_raw', 'smart_251_normalized', 'smart_251_raw', 'smart_252_normalized', 'smart_252_raw', 'smart_255_normalized', 'smart_255_raw']


Imprimimos las columnas y los elementos que más aparecen para hacernos una idea de que que haremos con los distintos datos.

In [6]:
for column in df.columns:
    print(f"Column: {column}")
    print(f"Top 3 most occurring elements: {df[column].value_counts().head(3).to_dict().keys()}")
    print(f"Number of NaN values: {df[column].isna().sum()}")
    print("\n")

Column: date
Top 3 most occurring elements: dict_keys(['2024-07-01', '2024-07-04', '2024-07-08'])
Number of NaN values: 0


Column: serial_number
Top 3 most occurring elements: dict_keys(['PL1331LAGXN3AH', '8160A0RSFVKG', '8160A0YNFVKG'])
Number of NaN values: 0


Column: model
Top 3 most occurring elements: dict_keys(['TOSHIBA MG08ACA16TA', 'TOSHIBA MG07ACA14TA', 'ST16000NM001G'])
Number of NaN values: 0


Column: capacity_bytes
Top 3 most occurring elements: dict_keys([-2147483648, 0, -1073741824])
Number of NaN values: 0


Column: failure
Top 3 most occurring elements: dict_keys([False, True])
Number of NaN values: 0


Column: smart_1_normalized
Top 3 most occurring elements: dict_keys([100.0, 83.0, 82.0])
Number of NaN values: 6574


Column: smart_1_raw
Top 3 most occurring elements: dict_keys([0.0, 65536.0, 1.0])
Number of NaN values: 6574


Column: smart_2_normalized
Top 3 most occurring elements: dict_keys([100.0, 132.0, 148.0])
Number of NaN values: 997203


Column: smart_2_raw

Podemos ver que las columnas con variables categóricas no tienen variables faltantes, vamos a trabajar solo reemplazando datos faltantes con las variables numéricas que pertenecen a los atributos S.M.A.R.T, hay normalizadas y crudas.

In [7]:
for column in df.columns:
    nan_count = df[column].isna().sum()
    if nan_count > 0:
        print(f"Column: {column}, NaN values: {nan_count}")

Column: smart_1_normalized, NaN values: 6574
Column: smart_1_raw, NaN values: 6574
Column: smart_2_normalized, NaN values: 997203
Column: smart_2_raw, NaN values: 997203
Column: smart_3_normalized, NaN values: 30328
Column: smart_3_raw, NaN values: 30328
Column: smart_4_normalized, NaN values: 30328
Column: smart_4_raw, NaN values: 30328
Column: smart_5_normalized, NaN values: 20813
Column: smart_5_raw, NaN values: 20813
Column: smart_7_normalized, NaN values: 30328
Column: smart_7_raw, NaN values: 30328
Column: smart_8_normalized, NaN values: 997203
Column: smart_8_raw, NaN values: 997203
Column: smart_9_normalized, NaN values: 4023
Column: smart_9_raw, NaN values: 4023
Column: smart_10_normalized, NaN values: 30328
Column: smart_10_raw, NaN values: 30328
Column: smart_11_normalized, NaN values: 2597935
Column: smart_11_raw, NaN values: 2597935
Column: smart_12_normalized, NaN values: 4023
Column: smart_12_raw, NaN values: 4023
Column: smart_13_normalized, NaN values: 2599798
Column: 

# Imputación de datos faltantes
----------
[Tratamiento de datos faltantes](D:/Downloads/Tratamiento_de_datos_faltantes_PDF.pdf)

La moda se usa para imputar predictores cualitativos (Variables categóricas) y el promedio o la mediana se usa para imputar predictores
cuantitativos (Variables numéricas).

### Procesamiento de variables categóricas

Variables categóricas con elementos NaN del dataset: ninguna

In [8]:
nan_counts = df.iloc[:, :5].isna().sum()
print(nan_counts)

date              0
serial_number     0
model             0
capacity_bytes    0
failure           0
dtype: int64


### Variables numéricas
Evaluamos la distribución:

In [9]:
output_dir = '../reports/figures/histogramas/'
os.makedirs(output_dir, exist_ok=True)

# Crear histogramas para las columnas 6 en adelante
for column in df.columns[6:]:
    plt.figure(figsize=(10, 6))
    df[column].dropna().hist(bins=50)
    plt.title(f'Histograma de {column}')
    plt.xlabel(column)
    plt.ylabel('Frecuencia')
    plt.savefig(f'{output_dir}{column}_histograma.png')
    plt.close()

#### Analizamos en búsqueda de datos atípicos:

Con el IQR vemos que columnas tienen anomalías y las guardamos en un diccionario

In [ ]:
# Calcular el rango intercuartílico (IQR) para las columnas 6 en adelante
iqr_values = {}
outliers_dict = {}
for column in df.columns[6:]:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    iqr_values[column] = IQR
    outliers = df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]
    print(f"Outliers in column {column}: {outliers.shape[0]}")
    if not outliers.empty:
        outliers_dict[column] = outliers

Outliers in column smart_1_raw: 177159
Outliers in column smart_2_normalized: 12313
Outliers in column smart_2_raw: 68
Outliers in column smart_3_normalized: 167137
Outliers in column smart_3_raw: 0
Outliers in column smart_4_normalized: 2249
Outliers in column smart_4_raw: 150874
Outliers in column smart_5_normalized: 9266
Outliers in column smart_5_raw: 98674
Outliers in column smart_7_normalized: 15583
Outliers in column smart_7_raw: 150438
Outliers in column smart_8_normalized: 1530
Outliers in column smart_8_raw: 94887
Outliers in column smart_9_normalized: 0
Outliers in column smart_9_raw: 63612
Outliers in column smart_10_normalized: 1710
Outliers in column smart_10_raw: 99
Outliers in column smart_11_normalized: 288
Outliers in column smart_11_raw: 252
Outliers in column smart_12_normalized: 1494
Outliers in column smart_12_raw: 148062
Outliers in column smart_13_normalized: 9
Outliers in column smart_13_raw: 45
Outliers in column smart_16_normalized: 0
Outliers in column smart

Analizamos en búsqueda de valores extremos:

# Codificación de variables categóricas
-------------
[Codificación de variables categóricas](D:/Downloads/codificacion_var_categoricas.pdf)

Variables que convertiremos a una variable categórica:



# Escalamiento